In [1]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
from datetime import datetime

%matplotlib inline

In [6]:
#intializing alpaca trade api
load_dotenv()
alpaca_api_key=os.getenv('ALPACA_API_KEY')
alpaca_secret_key=os.getenv('ALPACA_SECRET_KEY')
alpaca=tradeapi.REST(alpaca_api_key,alpaca_secret_key, api_version='v2')

In [102]:


#function that pulls tickers daily closing prices in the given time period using alpaca trade api, 
#intialization of the api is required before using.
#It returns a dataframe with the tickers, and closing prices as two level column structured datafram with index defined as date  

def get_tickers_data(tickers, start_date, end_date, timeframe):
    start_date = pd.Timestamp(start_date, tz = "America/New_York").isoformat()
    end_date = pd.Timestamp(end_date, tz = "America/New_York").isoformat()
    tickers_data_df = alpaca.get_bars(tickers, timeframe,start_date , end_date, adjustment = 'raw').df
    tickers_data_df.index = tickers_data_df.index.date
    tickers_data_df = tickers_data_df[['symbol', 'close']]
    tickers_data_list=[]
    for ticker in tickers:
        tickers_data_list.append(tickers_data_df[tickers_data_df['symbol']==ticker].drop('symbol', axis=1))
        
    clean_tickers_data = pd.concat(tickers_data_list,axis=1, keys=tickers)
    return clean_tickers_data

    



In [103]:
#defining the stockes in the solar energy portfolio form top performers in 2021 using mootley fool and investopedia. The companies are:
#1.First Solar (FSLR)
#2.Brookfield Renewable (BEPC)
#3.Solar Edge Technologies (SEDG)
#4.Daqo New Energy Corp. (DQ)
#5.Canadian Solar INC (CSIQ)

tickers = ['SPY', 'FSLR', 'BEPC' , 'SEDG', 'DQ','CSIQ']
timeframe='1Day'

start_date_war = '2022-02-20'
end_date_war = '2022-04-11'


tickers_data_wartime = get_tickers_data(tickers, start_date_war, end_date_war, timeframe) 


In [104]:
# COVID time frame (~2 years)
start_date_covid = '2020-01-01'
end_date_covid = '2022-02-19'
tickers_data_covid = get_tickers_data(tickers, start_date_covid, end_date_covid, timeframe) 

In [105]:
# pre-Covid time frame (2 years)
start_date_pre_covid = '2018-01-01'
end_date_pre_covid = '2019-12-31'
tickers_data_pre_covid = get_tickers_data(tickers, start_date_pre_covid, end_date_pre_covid, timeframe) 

In [106]:
#table= pd.pivot_table(yearly, values = 'close', index = [yearly.index], columns = ['symbol'])))
tickers_data_covid.head()

,SPY,FSLR,BEPC,SEDG,DQ,CSIQ
,close,close,close,close,close,close
2020-01-02,324.87,57.24,NaN,101.81,54.98,22.520
2020-01-03,322.41,57.02,NaN,102.38,54.45,21.800
2020-01-06,323.64,55.83,NaN,99.63,53.85,22.320
2020-01-07,322.73,56.67,NaN,101.79,53.36,22.355
2020-01-08,324.45,56.70,NaN,104.16,54.38,22.570
